<a href="https://colab.research.google.com/github/robin9804/Openholo_Val/blob/master/RS_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

!pip install plyfile
import plyfile

# RS method 검증 과정

1) point cloud reading

2) RS propagation -> complex image 추출

3) Encoding - openholo 사용 가능


In [37]:
# parameters
mm = 10e-3
um = mm*mm
nm = um*mm
wvl_R = 639 * nm  # Red
wvl_G = 525 * nm  # Green
wvl_B = 463 * nm  # Blue

def delta(wvl):
    return wvl * 10  # sampling period

def k0(wvl):
    return (np.pi * 2) / wvl  # wave number

pp = 3.45 * um  # pixel to pixel parameter

# resolution setting
width = 1080 
height = 1080

# PLY 파일 구성

ply  
format ascii 1.0  
comment Point Cloud Data Format in OpenHolo Library v1.0  
element color 1  
property int channel  
element vertex 7493  
property float x  
property float y  
property float z  
property uchar red  
property uchar green  
property uchar blue  
property double phase  
end_header


1번 항목이 꼭지점(Vertex), 7493개 꼭지점. x, y, z, Red, Green, Blue, phase로 구성되어있음


In [38]:
# read PLY file and convert to numpy array
with open('drive/My Drive/Colab Notebooks/data/dice_100000.ply', 'rb') as f:
    plydata = plyfile.PlyData.read(f)
data = np.array(plydata.elements[1].data)

In [39]:
# PLY data 검증
#np.exp(1j*data['phase'][0])
#plydata.elements[0]
data['x'][1]

0.10625

# R-S diffraction with Anti-alliasing

x, y 범위를 x0 - abs(tx / sqrt(1-tx**2)) < x 로 가줘야 한다.

결국 프레넬 회절 공식은 이러한 범위에 의해 zero padding된 형태로 남게 된다.


In [40]:
# define R-S method impulse response and Fresnel Integral method
def h_RS(x1, y1, x2, y2, z, wvl):
    """
    RS impulse response
    """
    r = np.sqrt((x1-x2)**2 + (y1-y2)**2 + z**2)
    h = (z / (1j * wvl)) * (np.exp(1j*k0(wvl)*r) / r **2)
    return h

def h_Fresnel(x, y, z, wvl):
    """
    Fresnel integral inpulse response
    """
    h = (np.exp(1j*k*z)/ 1j*wvl*z) * np.exp(1j*(k0(wvl)/(2*z))*(x**2 + y**2))
    return h

# tx, ty for anti alliasing 
def anti(wvl, z, p):
    txy = wvl / (2 * p)
    t = (txy / np.sqrt(1 - txy ** 2)) * z
    return np.abs(t)

def RS_kernel_conv(n, x2, y2, z2, wvl):
    x1 = data['x'][n]
    y1 = data['y'][n]
    z1 = data['z'][n]
    if wvl == wvl_R: 
        return data['red'][n]* h_RS(x1, y1,x2, y2, z1 - z2, wvl)
    elif wvl == wvl_G:
        return data['green'][n]* h_RS(x1, y1,x2, y2, z1 - z2, wvl)
    elif wvl == wvl_B:
        return data['blue'][n]* h_RS(x1, y1,x2, y2, z1 - z2, wvl)
    else:
        print("wrong number")

# RS method 를 사용한 hologram generation

RS method를 사용하여 홀로그램을 생성한다.

In [70]:
# R-S method
z1 = 0.5
phase = data['phase'][0]  # 5.059415

# 진입점
def RS_hologen(z, wvl):
    """
    hologram generation function
    """
    RS_in = np.zeros((width,height))  # hologram plane 생성
    for n in range(4, 5):
        RS_cash = np.zeros((width,height))
        for i in range(width):
            x2 = ((i-width)/2)* 4 * um # pixel location
            if data['x'][n] - anti(wvl, z, pp) < x2 < data['x'][n] +anti(wvl, z, pp):  # anti alliasing 조건
                for j in range(height):
                    y2 = ((j-height)/2) * 4 * um 
                    if data['y'][n] - anti(wvl, z, pp) < y2 < data['y'][n] +anti(wvl, z, pp):
                        RS_cash[i, j] =  1# RS_kernel_conv(n,x2, y2, z, wvl)
                    else:
                        pass
            else:
                pass
        RS_in = RS_in + RS_cash
        print(n , " th cash done")
    return RS_in

# Reconstruction image with ASM

angular spectrum methods를 사용하여 이미지를 다시 복원하는 과정

In [ ]:
# Reconst image with ASM
res = height / 2
def asm_kernel(Reconst_z):
    """
    Angular spectrum kernel to reconst picture
    """
    kernal = np.zeros((height // 2, height // 2))
    asm = -2j * np.pi * Reconst_z
    for i in np.arange(height // 2):
        for j in np.arange(height // 2):
            c = (-pp * res / 2 + pp / 2) + i * pp
            r = (-pp * res / 2 + pp / 2) + j * pp
            squrt = wvl ** -2 - ((r / pp - 0.5) / pp / res) ** 2 - ((c / pp - 0.5) / pp / res) ** 2
            kernal[[j], [i]] = np.exp(asm * np.sqrt(squrt))
    return kernal

def Reconst(z, ch):
    """
    Reconstruct hologram at z mm
    """
    p = asm_kernel(z)
    A = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(ch)))
    img = p * A
    img = np.fft.fftshift(np.fft.ifft2(np.fft.fftshift(img)))
    img = img * np.conj(img)
    img = img / np.max(img)
    #img to uint8
    return img